In [143]:
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
from matplotlib import pyplot as plt
import numpy as np
import utils
import numpy as np
from torch_geometric.utils.convert import from_networkx
import torch
from torch.nn import Linear
import torch.nn as nn
from torch_geometric.nn import GCNConv

In [144]:
g = nx.erdos_renyi_graph(100, 0.1, seed = 123)
model = ep.SIRModel(g)

# Model Configuration
config = mc.Configuration()
config.add_model_parameter('beta', 0.006)
config.add_model_parameter('gamma', 0.01)
config.add_model_parameter("fraction_infected", 0.1)
model.set_initial_status(config)

iterations = model.iteration_bunch(30)
trends = model.build_trends(iterations)

utils.print_graph_info(g)

Directed graph: False
Number of nodes: 100
Number of edges: 502


In [145]:
num_classes = 3
num_features = 3

x = nn.functional.one_hot(torch.tensor(list(model.status.values())), num_features).float()
y = nn.functional.one_hot(torch.tensor(list(model.initial_status.values())), num_features).float()

data = from_networkx(g)

In [146]:
class GCN(torch.nn.Module):
  def __init__(self):
    super(GCN, self).__init__()
    torch.manual_seed(42)
    self.conv1 = GCNConv(num_features, 3)
    self.conv2 = GCNConv(3, 6)
    self.conv3 = GCNConv(6, 3)
    self.conv4 = GCNConv(3, 3)
    self.conv5 = GCNConv(3, 3)
    self.conv6 = GCNConv(3, 3)
    self.conv7 = GCNConv(3, 3)
    self.classifier = Linear(3, num_classes)
  def forward(self, x, edge_index):
    h = self.conv1(x, edge_index)
    h = h.tanh()
    h = self.conv2(h, edge_index)
    h = h.tanh()
    h = self.conv3(h, edge_index)
    h = h.tanh()
    h = self.conv4(h, edge_index)
    h = h.tanh()
    h = self.conv5(h, edge_index)
    h = h.tanh()
    h = self.conv6(h, edge_index)
    h = h.tanh()
    h = self.conv7(h, edge_index)
    out = self.classifier(h)
    return out, h
  
model = GCN()

In [147]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
def train(data, x, y):
  optimizer.zero_grad()
  out, h = model(x, data.edge_index)
  loss = criterion(out, y)
  loss.backward()
  optimizer.step()
  return loss, h
epochs = range(1, 300)
losses = []
embeddings = []
for epoch in epochs:
  loss, h = train(data, x, y)
  losses.append(loss)
  embeddings.append(h)
  print(f"Epoch: {epoch}\tLoss: {loss:.4f}")

Epoch: 1	Loss: 1.2846
Epoch: 2	Loss: 1.1764
Epoch: 3	Loss: 0.9923
Epoch: 4	Loss: 0.8212
Epoch: 5	Loss: 0.6944
Epoch: 6	Loss: 0.5792
Epoch: 7	Loss: 0.4702
Epoch: 8	Loss: 0.3905
Epoch: 9	Loss: 0.3619
Epoch: 10	Loss: 0.3758
Epoch: 11	Loss: 0.4000
Epoch: 12	Loss: 0.4107
Epoch: 13	Loss: 0.4013
Epoch: 14	Loss: 0.3755
Epoch: 15	Loss: 0.3438
Epoch: 16	Loss: 0.3268
Epoch: 17	Loss: 0.3474
Epoch: 18	Loss: 0.3732
Epoch: 19	Loss: 0.3551
Epoch: 20	Loss: 0.3303
Epoch: 21	Loss: 0.3267
Epoch: 22	Loss: 0.3367
Epoch: 23	Loss: 0.3466
Epoch: 24	Loss: 0.3495
Epoch: 25	Loss: 0.3447
Epoch: 26	Loss: 0.3356
Epoch: 27	Loss: 0.3275
Epoch: 28	Loss: 0.3255
Epoch: 29	Loss: 0.3304
Epoch: 30	Loss: 0.3359
Epoch: 31	Loss: 0.3354
Epoch: 32	Loss: 0.3300
Epoch: 33	Loss: 0.3257
Epoch: 34	Loss: 0.3254
Epoch: 35	Loss: 0.3277
Epoch: 36	Loss: 0.3300
Epoch: 37	Loss: 0.3307
Epoch: 38	Loss: 0.3295
Epoch: 39	Loss: 0.3273
Epoch: 40	Loss: 0.3255
Epoch: 41	Loss: 0.3251
Epoch: 42	Loss: 0.3261
Epoch: 43	Loss: 0.3274
Epoch: 44	Loss: 0.32